### Pre Processing

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('prix_voiture.csv')

In [3]:
df.columns

Index(['etat_de_route', 'carburant', 'turbo', 'nombre_portes', 'type_vehicule',
       'roues_motrices', 'emplacement_moteur', 'empattement', 'longueur',
       'largeur', 'hauteur', 'poids_vehicule', 'type_moteur',
       'nombre_cylindres', 'taille_moteur', 'systeme_carburant',
       'taux_alésage', 'course', 'taux_compression', 'chevaux', 'tour_moteur',
       'consommation_ville', 'consommation_autoroute', 'prix', 'marque',
       'modèle'],
      dtype='object')

In [12]:
df["carburant"].unique()
df["modèle"].nunique()


141

In [13]:
df

,etat_de_route,carburant,turbo,nombre_portes,type_vehicule,roues_motrices,emplacement_moteur,empattement,longueur,largeur,...,taux_alésage,course,taux_compression,chevaux,tour_moteur,consommation_ville,consommation_autoroute,prix,marque,modèle
0,3,essence,atmosphérique,deux,convertible,propulsion,avant,2.25,4.28752,1.62814,...,88.138,68.072,9.0,111,5000,11.200667,8.711630,13495.0,alfa-romero,giulia
1,3,essence,atmosphérique,deux,convertible,propulsion,avant,2.25,4.28752,1.62814,...,88.138,68.072,9.0,111,5000,11.200667,8.711630,16500.0,alfa-romero,stelvio
2,1,essence,atmosphérique,deux,berline compacte,propulsion,avant,2.40,4.34848,1.66370,...,68.072,88.138,9.0,154,5000,12.379684,9.046692,16500.0,alfa-romero,Quadrifoglio
3,2,essence,atmosphérique,quatre,berline,traction,avant,2.53,4.48564,1.68148,...,81.026,86.360,10.0,102,5500,9.800583,7.840467,13950.0,audi,100 ls
4,2,essence,atmosphérique,quatre,berline,quatre roues motrices,avant,2.52,4.48564,1.68656,...,81.026,86.360,8.0,115,5500,13.067444,10.691545,17450.0,audi,100ls
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,-1,essence,atmosphérique,quatre,berline,propulsion,avant,2.77,4.79552,1.75006,...,96.012,80.010,9.5,114,5400,10.226696,8.400500,16845.0,volvo,145e (sw)
199,-1,essence,turbo,quatre,berline,propulsion,avant,2.77,4.79552,1.74752,...,96.012,80.010,8.7,160,5300,12.379684,9.408560,19045.0,volvo,144ea
200,-1,essence,atmosphérique,quatre,berline,propulsion,avant,2.77,4.79552,1.75006,...,90.932,72.898,8.8,134,5500,13.067444,10.226696,21485.0,volvo,244dl
201,-1,diesel,turbo,quatre,berline,propulsion,avant,2.77,4.79552,1.75006,...,76.454,86.360,23.0,106,4800,9.046692,8.711630,22470.0,volvo,246


In [11]:
# Define X and y variables
y = df['prix']
X = df.drop(['prix'], axis=1)
###############################################################################################
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)
###############################################################################################
# Les colonnes numériques et catégorielles
categorial_features = ['etat_de_route', 'carburant','turbo','nombre_portes', 'type_vehicule', 'roues_motrices', 'emplacement_moteur','type_moteur', 'nombre_cylindres', 'systeme_carburant', 'marque', 'modele']
numeric_features = ['empattement', 'longueur_voiture' , 'largeur_voiture', 'poids_vehicule', 'taille_moteur', 'taux_alésage', 'chevaux', 'consommation_ville', 'consommation_autoroute']
# numerical_columns = ['empattement', 'longueur_voiture', 'largeur_voiture', 'poids_vehicule', 'taille_moteur', 'taux_alésage', 'chevaux', 'consommation_ville', 'consommation_autoroute']
# categorical_columns = ['etat_de_route', 'turbo', 'type_vehicule', 'roues_motrices', 'emplacement_moteur', 'marque', 'modele']
#############################################################################################
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, PolynomialFeatures

# On cree un transformateur pour les variables catégorielles
categorical_transformer = Pipeline(steps=[
    ('onehotencoder', OneHotEncoder(handle_unknown='ignore'))
])
# categorical_transformer = OneHotEncoder(sparse=True)

# On cree un transformateur pour les variables numériques
numeric_transformer = Pipeline([
        ('min_max', MinMaxScaler()),  # moyenne nulle et écart type = 1 -> Reg, SVM, PCA
        ('poly', PolynomialFeatures(degree=2, include_bias=False))
        ])
# Créer un préprocesseur pour combiner les deux transformateurs
# A l'aide de la classe ColumnTransformer, on déclare à quelles variables on applique quel transformer
from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorial_features)
    ]
)
################################################################################################
from sklearn.linear_model import LinearRegression
linear_regression_pipeline = Pipeline(steps=[    ('preprocessor', preprocessor),    ('regressor', LinearRegression())])
+
# from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import GridSearchCV
linear_param_grid = {
    'regressor__fit_intercept': [True, False]
}
# Ceci va chercher les deux valeurs possibles du paramètre fit_intercept et utiliser une validation croisée à 5 plis pour évaluer les performances du modèle.
# Le paramètre scoring est défini à 'r2', ce qui spécifie que nous voulons optimiser la métrique du coefficient de détermination (R²).
linear_grid = GridSearchCV(linear_regression_pipeline, param_grid=linear_param_grid, cv=5, scoring='r2')
linear_grid.fit(X_train, y_train)


['etat_de_route', 'carburant', 'turbo', 'nombre_portes', 'type_vehicule', 'roues_motrices', 'emplacement_moteur', 'type_moteur', 'nombre_cylindres', 'systeme_carburant', 'marque', 'modele']
['empattement', 'longueur_voiture', 'largeur_voiture', 'poids_vehicule', 'taille_moteur', 'taux_alésage', 'chevaux', 'consommation_ville', 'consommation_autoroute']
